In [25]:
from sklearn.model_selection import KFold
from glob import glob

tar_files = glob("data/train/*.tar")
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

train_idx, val_idx = list(kfold.split(tar_files))[0]

In [26]:
train_split, val_split = [tar_files[x] for x in train_idx], [tar_files[x] for x in val_idx]

In [27]:
train_split, val_split

(['data/train\\train-0000.tar',
  'data/train\\train-0002.tar',
  'data/train\\train-0003.tar',
  'data/train\\train-0004.tar',
  'data/train\\train-0005.tar',
  'data/train\\train-0006.tar',
  'data/train\\train-0007.tar',
  'data/train\\train-0009.tar'],
 ['data/train\\train-0001.tar', 'data/train\\train-0008.tar'])